In [2]:
import mxnet as mx
from mxnet import nd, init, gluon, autograd, metric
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms

import matplotlib.pyplot as plt
from time import time
%matplotlib inline

In [3]:
#Get data
mnist_train = gluon.data.vision.datasets.FashionMNIST(train=True)
X, y = mnist_train[0]
print("X_shape : %s dtype %s" %(X.shape, X.dtype))
print("Number of image: %d" %len(mnist_train))

X_shape : (28, 28, 1) dtype <class 'numpy.uint8'>
y_shape : () dtype int32
Number of image: 60000


In [8]:
X,y = mnist_train[0:6]

# Transform dataset
* channel first, float 32
* normalize

In [9]:
transform_fn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.13, 0.31)
])

In [10]:
mnist_train = mnist_train.transform_first(transform_fn)

In [12]:
# Data Loading
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True,num_workers=4)

In [14]:
for data, label in train_data:
    print(data.shape, label.shape)
    break

(256, 1, 28, 28) (256,)


# Define Model
* LeNet 5 model

In [17]:
net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Flatten(),
        nn.Dense(120, activation="relu"),
        nn.Dense(84, activation="relu"),
        nn.Dense(10)
    )
    
net

Sequential(
  (0): Conv2D(None -> 6, kernel_size=(5, 5), stride=(1, 1), Activation(relu))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (2): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (4): Flatten
  (5): Dense(None -> 120, Activation(relu))
  (6): Dense(None -> 84, Activation(relu))
  (7): Dense(None -> 10, linear)
)

In [18]:
net.initialize(init=init.Xavier())

In [21]:
# Loss
softmax_crossentropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [22]:
# Metric
train_acc = metric.Accuracy()

In [23]:
# Optimization
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [24]:
# Training Loop
for epoch in range(10):
    train_loss = 0.
    tic = time()
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_crossentropy(output, label)
            loss.backward()
            
        trainer.step(batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc.update(label, output)
        
    print("Epoch(%d) Loss: %.3f Acc: %.3f perf: %.1f img/sec"
          %(epoch, train_loss/len(train_data), train_acc.get()[1], len(mnist_train)/time()-tic))

Epoch(0) Loss: 0.833 Acc: 0.690 perf: -1582001429.3 img/sec
Epoch(1) Loss: 0.476 Acc: 0.755 perf: -1582001689.8 img/sec
Epoch(2) Loss: 0.408 Acc: 0.787 perf: -1582001876.5 img/sec
Epoch(3) Loss: 0.369 Acc: 0.807 perf: -1582002061.3 img/sec
Epoch(4) Loss: 0.345 Acc: 0.820 perf: -1582002251.1 img/sec
Epoch(5) Loss: 0.322 Acc: 0.830 perf: -1582002446.7 img/sec
Epoch(6) Loss: 0.310 Acc: 0.838 perf: -1582002626.2 img/sec
Epoch(7) Loss: 0.297 Acc: 0.845 perf: -1582002809.1 img/sec
Epoch(8) Loss: 0.285 Acc: 0.850 perf: -1582003004.0 img/sec
Epoch(9) Loss: 0.275 Acc: 0.855 perf: -1582003197.9 img/sec


In [25]:
net.save_parameters("trained_net.params")

# Neural Network Evaluation

In [26]:
# load parameters
net.load_parameters("trained_net.params")

In [27]:
# Predict
mnist_valid = datasets.FashionMNIST(train=False)

In [28]:
# Data Transformation
transform_fn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.13, 0.31)
])

In [29]:
# Predict for 6 images
preds = []
for idx in range(6):
    image,label = mnist_valid[idx]
    image = transform_fn(image).expand_dims(axis=0)
    pred = net(image).argmax(axis=1)
    preds.append(pred.astype('int32').asscalar())

In [31]:
# Validation dataloader

batch_size = 256
valid_data = gluon.data.DataLoader(
    mnist_valid.transform_first(transform_fn), 
    batch_size = batch_size
)

In [32]:
# Accuracy
valid_acc = metric.Accuracy()

In [33]:
# Validation Loop
for data, label in valid_data:
    output = net(data)
    valid_acc.update(label, output)
    print("Validation Acc: %.3f" %(valid_acc.get()[1]))

Validation Acc: 0.883
Validation Acc: 0.881
Validation Acc: 0.888
Validation Acc: 0.887
Validation Acc: 0.886
Validation Acc: 0.883
Validation Acc: 0.876
Validation Acc: 0.876
Validation Acc: 0.876
Validation Acc: 0.875
Validation Acc: 0.876
Validation Acc: 0.877
Validation Acc: 0.876
Validation Acc: 0.876
Validation Acc: 0.878
Validation Acc: 0.880
Validation Acc: 0.883
Validation Acc: 0.883
Validation Acc: 0.885
Validation Acc: 0.884
Validation Acc: 0.885
Validation Acc: 0.886
Validation Acc: 0.885
Validation Acc: 0.885
Validation Acc: 0.886
Validation Acc: 0.885
Validation Acc: 0.885
Validation Acc: 0.885
Validation Acc: 0.886
Validation Acc: 0.886
Validation Acc: 0.886
Validation Acc: 0.887
Validation Acc: 0.887
Validation Acc: 0.888
Validation Acc: 0.888
Validation Acc: 0.888
Validation Acc: 0.889
Validation Acc: 0.890
Validation Acc: 0.890
Validation Acc: 0.890
